# Agenda

1. The iterator protocol
    - Adding iteration to your classes
    - Different techniques for that
2. Generator functions
3. Generator comprehensions
4. Decorators
5. Threading and multiprocessing 